In [1]:
import pandas as pd
pd.options.display.max_columns = 200
import numpy as np
from pprint import pprint
import re

# Reading Data

In [95]:
df = pd.read_csv('DDS11_Data_Extract_with_labels.csv').replace("#NULL!", "")
#df = pd.read_csv('Preprocessed_data.csv')

df.head()

,record - Record number,"Q1r1 - To begin, what is your age?",Q4 - What is your gender?,age - you are...,Q2 - In which state do you currently reside?,region - Region,QNEW3 - What is your employment status?,Q5 - Which category best describes your ethnicity?,QNEW1 - Do you have children living in your home (excluding yourself if you are under 18)?,QNEW2 - How old are the children in your home?-0-4 years,QNEW2 - How old are the children in your home?-5-9 years,QNEW2 - How old are the children in your home?-10-13 years,QNEW2 - How old are the children in your home?-14-18 years,QNEW2 - How old are the children in your home?-19-25 years,QNEW2 - How old are the children in your home?-26+ years,QNEW2 - How old are the children in your home?-Don't Know,"Q6 - Into which of the following categories does your total annual household income fall before taxes? Again, we promise to keep this, and all your answers, completely confidential.",Q8 - Which of the following media or home entertainment equipment does your household own?-Flat panel television,Q8 - Which of the following media or home entertainment equipment does your household own?-Digital video recorder (DVR),Q8 - Which of the following media or home entertainment equipment does your household own?-Streaming media box or over-the-top box,Q8 - Which of the following media or home entertainment equipment does your household own?-Portable streaming thumb drive/fob,Q8 - Which of the following media or home entertainment equipment does your household own?-Over-the-air digital TV antenna (for free access to network broadcast without pay TV subscription),Q8 - Which of the following media or home entertainment equipment does your household own?-Blu-ray disc player/DVD player,Q8 - Which of the following media or home entertainment equipment does your household own?-Gaming console,Q8 - Which of the following media or home entertainment equipment does your household own?-Portable video game player,Q8 - Which of the following media or home entertainment equipment does your household own?-Computer network/router in your home for wireless computer/laptop usage,Q8 - Which of the following media or home entertainment equipment does your household own?-Desktop computer,Q8 - Which of the following media or home entertainment equipment does your household own?-Laptop computer,Q8 - Which of the following media or home entertainment equipment does your household own?-Tablet,Q8 - Which of the following media or home entertainment equipment does your household own?-Dedicated e-book reader,Q8 - Which of the following media or home entertainment equipment does your household own?-Smartphone,Q8 - Which of the following media or home entertainment equipment does your household own?-Basic mobile phone (not a smartphone),Q8 - Which of the following media or home entertainment equipment does your household own?-Smart watch,Q8 - Which of the following media or home entertainment equipment does your household own?-Fitness band,Q8 - Which of the following media or home entertainment equipment does your household own?-Virtual reality headset,Q8 - Which of the following media or home entertainment equipment does your household own?-Drone,Q8 - Which of the following media or home entertainment equipment does your household own?-None of the above,Q8 - Which of the following media or home entertainment equipment does your household own?-Don't Know,"Q10 - Of those products you indicated you do not currently own, which of the following do you plan to purchase in the next 12 months?-Flat panel television","Q10 - Of those products you indicated you do not currently own, which of the following do you plan to purchase in the next 12 months?-Digital video recorder (DVR)","Q10 - Of those products you indicated you do not currently own, which of the following do you plan to purchase in the next 12 months?-Streaming media box or over-the-top box","Q10 - Of those products you indicated you do not currently own, which of the followin

In [35]:
[col for col in df.columns if 'being the most preferred' in col]

['q37r1 - attending live performances (sporting events, concerts, or stage (musical, dramatic, or other)) - for the following types of entertainment activities, please rank your top three, with "1" being the most preferred.please rank the top three, with â€œ',
 'q37r2 - going to the movies - for the following types of entertainment activities, please rank your top three, with "1" being the most preferred.please rank the top three, with â€œ1â€\x9d being the most preferred. make your selections by clicking each item in',
 'q37r3 - watching television (video content on any device) - for the following types of entertainment activities, please rank your top three, with "1" being the most preferred.please rank the top three, with â€œ1â€\x9d being the most preferred. make your selec',
 'q37r4 - listening to music (using any device) - for the following types of entertainment activities, please rank your top three, with "1" being the most preferred.please rank the top three, with â€œ1â€\x9d bei

In [62]:
[col for col in df.columns if 'is your most' in col]

['Q89 - Which of the following is your most frequently used mechanism to get news?']

# Grouping Questions and their options

In [96]:
new_col_names = []

for name in df.columns:
    if 'news' not in name:
        new_col_names.append( name.lower().replace('new', '') )
    else:
        new_col_names.append( name.lower() )

In [97]:
new_col_names

['record - record number',
 'q1r1 - to begin, what is your age?',
 'q4 - what is your gender?',
 'age - you are...',
 'q2 - in which state do you currently reside?',
 'region - region',
 'q3 - what is your employment status?',
 'q5 - which category best describes your ethnicity?',
 'q1 - do you have children living in your home (excluding yourself if you are under 18)?',
 'q2 - how old are the children in your home?-0-4 years',
 'q2 - how old are the children in your home?-5-9 years',
 'q2 - how old are the children in your home?-10-13 years',
 'q2 - how old are the children in your home?-14-18 years',
 'q2 - how old are the children in your home?-19-25 years',
 'q2 - how old are the children in your home?-26+ years',
 "q2 - how old are the children in your home?-don't know",
 'q6 - into which of the following categories does your total annual household income fall before taxes? again, we promise to keep this, and all your answers, completely confidential.',
 'q8 - which of the followi

In [98]:
# Cleaning column names
df.columns = new_col_names

In [99]:
# create a dictionary to store the columns for each question number
questions = {}

# iterate over the columns
for col in df.columns:
    # use a regular expression to extract the question number
    match = re.search(r'q(\d+)', col)
    if match:
        question_num = match.group(1)
        # add the column to the list for the question number
        key = int(question_num)
        if key not in questions:
            questions[key] = []
        questions[key].append(col)

In [100]:
# Fixing id 1
questions[0], questions[1] = [questions[1][0]], [questions[1][-1]]

# Fixing id 2
state_question_index = 0
questions[-1] = [ questions[2][ state_question_index ] ] # setting it as -1
questions[2].pop( state_question_index ) # Removing in from q2 dict key

# Fixing id 29 (by splitting into different questions)
questions[30] = [ questions[29][1] ]
questions[29].pop(1)

"q29 - when you use a friend or family member's subscription login information to watch digital content, what type of content do you most often watch?"

In [101]:
questions.keys()

dict_keys([1, 4, 2, 3, 5, 6, 8, 10, 11, 15, 16, 17, 22, 26, 36, 29, 37, 19, 20, 24, 28, 73, 39, 89, 0, -1, 30])

In [102]:
df[questions[37]]

,"q37r1 - attending live performances (sporting events, concerts, or stage (musical, dramatic, or other)) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ","q37r2 - going to the movies - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most preferred. make your selections by clicking each item in","q37r3 - watching television (video content on any device) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most preferred. make your selec","q37r4 - listening to music (using any device) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most preferred. make your selections by cli","q37r5 - reading books (either physical books or via an e-book reader and/or online) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most","q37r6 - reading magazines (either printed or online) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most preferred. make your selections","q37r7 - reading newspapers (either printed or online) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most preferred. make your selection","q37r8 - listening to the radio (any format and/or device) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most preferred. make your selec","q37r9 - playing video games (handhelds, pc, console, mobile/cellular/smartphone, online) - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the","q37r10 - using the internet for social or personal interests - for the following types of entertainment activities, please rank your top three, with ""1"" being the most preferred.please rank the top three, with â€œ1â€ being the most preferred. make your se"
0,1,,,,,,2,3,,
1,,,,,3,,,2,,1
2,3,,,,,,2,,,1
3,,1,,2,,,,,3,
4,,,2,,,,,,1,3
...,...,...,...,...,...,...,...,...,...,...
2126,3,2,,,,,,,1,
2127,,,,2,3,,,,,1
2128,,,3,,,,,,1,2
2129,,,2,,3,1,,,,


# Cleaning Column names in sub-frames that use the question numbers and stacking

In [103]:
def stack_and_clean(df, col_names, question):
    # Renaming Columns
    df.columns = col_names

    # Stacking
    df = df.stack().to_frame().rename(columns={0:'Selection'})

    # Adding question
    df['Question'] = question

    # Chaging index name
    df.index.names = ['subject_nr', 'option']

    # Adding question type
    df['Q_selection_type'] = 'multiple' if (len(col_names)>1) else 'single'

    # Aditional Cleaning to
    df.index = df.index.set_levels(df.index.levels[1].str.lstrip('-'), level='option') #avoiding anything starting with "-", so it is cleaned
    df.index = df.index.set_levels(df.index.levels[1].str.rstrip('.'), level='option') #avoiding anything   ending with ".", so it is cleaned

    return df

In [104]:
frames = []

for qid, _wtf in questions.items():
#for qid, _wtf in [(11, questions[11])]:
    # Getting question names
    char1, char2 = '-', '?'
    shorting_question = [pregunta[pregunta.find(char1)+2 : pregunta.find(char2)] for pregunta in _wtf] # filtering between characters
    shorting_question = list( set( shorting_question ) ) # removing those that are repeated (probably multi selection questions)
    
    if len(shorting_question) > 1:
        #print( f"Repeated sentences in Q{qid}:\n{shorting_question}" )        
        # Splitting
        multi_selection_answers = [ senteces.split(' - ')[0] for senteces in shorting_question ]
        multi_selection_questions = list(set([ senteces.split(' - ')[1] for senteces in shorting_question ]))[0]
        
        # These are questions destined for agreement level. This would facilitate filtering
        if 'scale below' in multi_selection_questions:
            multi_selection_questions = 'Agreement Level'
            
    else:
        #print( f"Q{qid}:\n{shorting_question}" )
        if len(_wtf)>1:
            multi_selection_answers = [answ.split('?')[1] for answ in _wtf]
        else:
            # Keeping the filtered version above
            multi_selection_answers = [pregunta[pregunta.find(char1)+2 : pregunta.find(char2)] for pregunta in _wtf]#_wtf

        multi_selection_questions = shorting_question[0]
    
    # Filtering for the original column names
    fr = stack_and_clean(df[ _wtf ].copy(), multi_selection_answers, multi_selection_questions)
    
    # Appending dataframe
    frames.append(fr)

# Compiling
df_answers = pd.concat( frames, ignore_index=False )

In [88]:
df_answers['Selection'].unique()#[df_answers['Q_selection_type'] == 'single'].index.get_level_values(level='option').str.lstrip('-')#.unique()

array(['Yes', 'No', 'Male', 'Female', '',
       'Employed full-time or part-time', 'Student', 'Self-employed',
       'Retired', 'Unemployed', 'White or Caucasian (Non-Hispanic)',
       'African American', 'Hispanic',
       'South Asian (India, Pakistan, Sri Lanka)',
       'Something else not listed here', 'Caribbean',
       'East Asian or Southeast Asian',
       'Pacific Islander (including Native Hawaiian, Native American, or Native Alaskan)',
       'Multiracial', 'Middle Eastern', '$50,000 to $99,999',
       'Less than $29,999', '$100,000 to $299,999', '$30,000 to $49,999',
       'Do not know', 'More than $300,000', '3', '1', '2',
       'Asked Not Ranked', 0.0, 100.0, 40.0, 30.0, 10.0, 25.0, 15.0, 50.0,
       80.0, 20.0, 5.0, 70.0, 60.0, 90.0, 95.0, 75.0, 35.0, 33.0, 34.0,
       65.0, 8.0, 22.0, 1.0, 98.0, 55.0, 32.0, 85.0, 3.0, 86.0, 45.0,
       99.0, 2.0, 12.0, 88.0, 28.0, 17.0, 93.0, 4.0, 11.0, 6.0, 83.0,
       97.0, 62.0, 38.0, 9.0, 23.0, 51.0, 59.0, 31.0, 49.0, 19

In [105]:
df_answers.to_csv('Digital_Answers.csv', encoding='utf-8')

# Further inspection

In [109]:
df_answers.index.get_level_values(level='option').str.extract(r'([^a-z A-Z])', expand=False).value_counts()

/    55406
(    44751
-    25572
'    14917
1     6393
,     6393
0     2131
5     2131
2     2131
"     2131
.     2131
Name: option, dtype: int64

In [114]:
df_answers[df_answers.Question.str.contains('being the most')].iloc[0,1]

'for the following types of entertainment activities, please rank your top three, with "1" being the most preferred.please rank the top three, with â€œ1â€\x9d being the mos'